In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-08-08 13:45:34--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273 (4.2K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   4.17K  --.-KB/s    in 0s      

2025-08-08 13:45:34 (32.2 MB/s) - ‘minsearch.py’ saved [4273/4273]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/workspaces/llm-zoomcamp/0a-agents/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='deepseek-r1',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
# Local LLM
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [14]:
llm('write that this is a test')

'<think>\nOkay, the user wrote "write that this is a test" which seems like they\'re explicitly asking me to label something as a test. \n\nHmm, making sure I understand what they want here - it\'s literally asking for confirmation that whatever interaction we\'re having is just a simulation or practice scenario. There might be some meta-awareness at play where the user wants to acknowledge our artificial nature upfront.\n\nThe phrasing is minimal but precise. They didn\'t say "I know this is AI" or anything similar, but they did choose language that points toward testing. Maybe they\'re checking how I handle self-references in a professional context? \n\nSince they\'ve given me no additional text or scenario to work with beyond the base prompt itself, keeping it straightforward makes sense. No need to overcomplicate - just deliver exactly what was requested while maintaining appropriate formality based on their language style.\n\nThe user seems technically savvy enough to craft phrase

In [19]:
print(_)

<think>
Okay, the user wrote "write that this is a test" which seems like they're explicitly asking me to label something as a test. 

Hmm, making sure I understand what they want here - it's literally asking for confirmation that whatever interaction we're having is just a simulation or practice scenario. There might be some meta-awareness at play where the user wants to acknowledge our artificial nature upfront.

The phrasing is minimal but precise. They didn't say "I know this is AI" or anything similar, but they did choose language that points toward testing. Maybe they're checking how I handle self-references in a professional context? 

Since they've given me no additional text or scenario to work with beyond the base prompt itself, keeping it straightforward makes sense. No need to overcomplicate - just deliver exactly what was requested while maintaining appropriate formality based on their language style.

The user seems technically savvy enough to craft phrased intended as te